In [1]:
import pandas as pd
#from catboost import Pool, CatBoostClassifier
import shap

import sklearn.metrics as m
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.compose import make_column_transformer
from sklearn.feature_extraction.text import TfidfVectorizer

from imblearn.over_sampling import SMOTE
from imblearn.pipeline import make_pipeline

import spacy
import ru_core_news_sm
nlp = spacy.load('ru_core_news_sm')

In [2]:
df = pd.read_csv('cld.csv')

In [4]:
df.columns


Index(['admittion', 'discharge', 'sex', 'height', 'weight', 'BMI', 'BSA',
       'birth', 'Age', 'Операции (все в ИБ)', 'Перенесенные опер. (из Анамн.)',
       'redo', 'Диагноз', 'МКБ', 'Соп. забол. (из Анамн.)',
       'Медикам. леч. по поводу осн. забол. (из Анамн.)',
       'Принимаемые препараты (из Анамн.)', 'ЭхоКГ (Из Эпикр. до опер.)',
       'ЭКГ (Из Эпикр. до опер.)', 'АКГ (Из Эпикр. до опер.)',
       'КТ (Из Эпикр. до опер.)', 'Операция (основная / первая в ИБ)',
       'Дата опер.', 'DateOfOper', 'target'],
      dtype='object')

In [6]:
df[['Операция (основная / первая в ИБ)', 'Операции (все в ИБ)']]

,Операция (основная / первая в ИБ),Операции (все в ИБ)
0,Перевязка ранее наложенного анастомоза по Blal...,12.12.2016: (Откр./ИК) Перевязка ранее наложен...
1,Радикальная коррекция двойного отхождения маги...,18.01.2017: (Откр./ИК) Радикальная коррекция д...
2,Перевязка ранее наложенного анастомоза по Blal...,19.01.2017: (Откр./ИК) Перевязка ранее наложен...
3,Наложение двустороннего кавапульмонального ана...,23.01.2017: (Откр./ИК) Наложение двустороннего...
4,Реконструкция путей оттока из правого желудочк...,15.02.2017: (Откр./ИК) Реконструкция путей отт...
...,...,...
295,Радикальная коррекция частично открытого атрио...,27.09.2017: (Откр./ИК) Радикальная коррекция ч...
296,"Операция Muller, суживание легочной артерии","02.02.2017: (Закрыт.) Операция Muller, суживан..."
297,Наложение модифицированного подключично-легочн...,27.01.2017: (Закрыт.) Наложение модифицированн...
298,Перевязка ранее наложенного анастомоза по Blal...,02.08.2017: (Откр./ИК) Перевязка ранее наложен...


In [9]:
df[1] = df['Операция (основная / первая в ИБ)'].str.len()
df[2] = df['Операции (все в ИБ)'].str.len()

In [13]:
df['dif'] = df[2] - df[1]
df['dif'].value_counts()

23     174
22      55
24      47
128      2
149      2
71       2
120      1
190      1
114      1
156      1
147      1
133      1
276      1
157      1
258      1
113      1
115      1
312      1
21       1
82       1
79       1
72       1
214      1
65       1
Name: dif, dtype: int64

In [3]:
df.columns = ['admittion', 'discharge', 'sex', 'height', 'weight', 'BMI', 'BSA', 'birth',
              'age', 'opertatzii', 'before_op', 'redo', 'diag', 'mkb', 'soput',
              'medic_osn', 'medic', 'eho', 'ekg', 'akg', 'kt', 'first_op',
              'op_date', 'DateOfOper', 'target']
df['id'] = df.index

df['redo'] = df['redo'].fillna(0)
#df = df.fillna('')

In [4]:
df.tail(2)

,admittion,discharge,sex,height,weight,BMI,BSA,birth,age,opertatzii,...,medic,eho,ekg,akg,kt,first_op,op_date,DateOfOper,target,id
298,2017-07-27,2017-08-11,m,65,6.80,84.34,0.35,12.01.2017,202,02.08.2017: (Откр./ИК) Перевязка ранее наложен...,...,"аспирин-кардио, дигоксин, панангин, элькар, ви...","ЭхоКГ ВПС (27.07.2017 13:46:33, Г.)\nЛЖ, ЛП:...",ЭКГ (27.07.2017 15:58:11)\nРитм сердца синусов...,АКГ ВПС (28.07.2017 13:35:25): Заключение: Дво...,NaN,Перевязка ранее наложенного анастомоза по Blal...,02.08.2017 9:50:30,2017-08-02,0,298
299,2017-05-11,2017-05-22,f,64,5.42,67.75,0.31,06.12.2016,160,15.05.2017: (Откр./ИК) Пластика дефекта межжел...,...,"аспаркам, верошпирон, дигоксин","от 27.04.2017 Левое предсердие, левый желудоче...","от 11.05.2017 Ритм правильный, синусовый, ЭОс ...",NaN,NaN,Пластика дефекта межжелудочковой перегородки к...,15.05.2017 8:40:00,2017-05-15,0,299


_______________________

____________
# EHO part

In [5]:
eho = df[['id', 'eho']].copy()

In [6]:
eho['eho'] = eho['eho'].str.lower()

In [7]:
eho['eho_'] = eho['eho'].str.extract(r'заключение:? ?(.*?)$').fillna(' ').astype('str')

In [8]:
res = pd.Series(eho['eho_'].sum().split(' ')).value_counts() 
res[res > 20]

                   86
увеличение         62
стеноз             57
отделов            51
дмжп,              46
гипоплазия         44
с                  39
правых             36
в                  35
ла,                33
мм                 31
мпс,               30
ла                 28
на                 28
оап,               28
ооо,               27
клапанный          25
левых              25
п/о                25
и                  24
легочной           23
аорты,             22
функционирующий    22
dtype: int64

In [9]:
eho_list = ['стеноз', 'гипоплазия', 'мпс', 'ооо', 'оап', 'дмжп', 'анастомоз']

In [10]:
for col in eho_list:
    eho[col] = eho['eho'].str.contains(col, na=False).astype('int')

In [11]:
eho = eho[['id', *eho_list]].fillna(0)

________________________________
## MKB part

In [12]:
mkb = df[['id', 'mkb']]

In [13]:
mkb['mkb'] = mkb['mkb'].str.split(',')
mkb = mkb.explode('mkb')


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [14]:
mkb['mkb'] = mkb['mkb'].str.replace('Q','',regex=True).str.strip().astype('str')

In [15]:
mkb_list = mkb['mkb'].value_counts()
mkb_list = mkb_list[mkb_list > 2].index.to_list()

In [16]:
mkb.loc[~mkb['mkb'].isin(mkb_list),'mkb'] = 'Oth'

In [17]:
mkb['val'] = 1

In [18]:
mkb = mkb.pivot_table(columns='mkb', index='id', values='val').reset_index().fillna(0).astype('int')

__________________________
## Drugs part

In [19]:
medic = df[['id', 'medic_osn','medic']]

In [20]:
medic = medic.fillna('')

In [21]:
medic['medic_osn'] += ' ' + medic['medic']

In [22]:
medic['medic_osn'] = medic['medic_osn'].str.replace(r'[^а-яА-Я -]', ' ', regex=True) \
                                       .str.replace(' +', ' ', regex=True).str.lower().str.strip()

In [23]:
# drugs = medic['medic_osn'].map(lambda x: set(x.split(' '))).explode().value_counts()

In [24]:
drugs = set(['верошпирон', 'дигоксин', 'фуросемид', 'аспаркам', 'капотен', 'элькар',
         'панангин', 'аспирин', 'вазапростан', 'анаприлин', 'допамин', 'адреналин',
         'аспирин-кардио','мочегонные', 'алпростан', 'гепарин',
         'ампициллин', 'нанипрус', 'траклир', 'гентамицин'])

In [25]:
medic['medic_osn'] = medic['medic_osn'].map(lambda x: set(x.split(' ')) & drugs)

In [26]:
medic = medic.explode('medic_osn')

In [27]:
medic_table = medic.pivot_table(index='id', columns='medic_osn', aggfunc='count', values='medic_osn').reset_index().fillna(0)

In [28]:
medic_table.columns = ['id', *(f'drug_{i}' for i, _ in enumerate(medic_table.columns[1:]))]

__________________________
## EKG part

In [29]:
ekg = df[['ekg','id']].copy()
ekg = ekg.dropna()
ekg['ekg'] = ekg['ekg'].str.lower()

In [30]:
# дата, синусовый
ekg['ekg_date'] = ekg['ekg'].str.extract(r'(\d+\.\d+\.\d{2,4})')
ekg['ekg_sinus'] = ekg['ekg'].str.extract(r'( синус)').notna().astype('int')

In [31]:
# ЧСС 
list_of_ss = '|'.join(['чсс', 'сердцебиений в минуту', 'сердечных сокращений'])
res = ekg['ekg'].str.extract(f'({list_of_ss}) ?=?-? ?(\d+)')
ekg['ekg_ss'] = res[1]

In [32]:
# эос
eos_dict = {'прав': 1,
            'лев': 2,
            'норм':3, 
            'гориз':4, 
            'верт':5}

def ekg_replace(data):
    res = [eos_dict.get(x) for x in eos_dict.keys() if x in data]
    if res:
        return res[0]
    return 0

list_of_eos = '|'.join(['эос', 'электрической оси сердца:', 'ось сердца', 'ось'])
res = ekg['ekg'].str.extract(f'(({list_of_eos}) (отклонена)? ?(\w+))|((\w+) (положение э))')
res[3] = res[3].fillna(res[5])
ekg['ekg_eos'] = res[3]
ekg['ekg_eos'] = ekg['ekg_eos'].fillna(' ')
ekg['ekg_eos'] = ekg['ekg_eos'].map(ekg_replace)

In [33]:
ekg['ekg_qrs'] = ekg['ekg'].str.extract('(qrs) ?=? ?(\d(,|\.)\d+)')[1].str.replace(',','.', regex=True).astype('float')
ekg['ekg_qrst'] = ekg['ekg'].str.extract('(qrst) ?=? ?(\d(,|\.)\d+)')[1].str.replace(',','.', regex=True).astype('float')
ekg['ekg_pq'] = ekg['ekg'].str.extract('(pq) ?=? ?(\d(,|\.)\d+)')[1].str.replace(',','.', regex=True).astype('float')

In [34]:
#ekg['ekg_clr'] = ekg['ekg'].str.lower().str.replace(r'[^а-я ]', ' ', regex=True)
#ekg['ekg_clr'] = ekg['ekg_clr'].str.replace(r'\b\w{,3}\b',' ',regex=True).str.replace(r' +',' ', regex=True).str.strip()

In [35]:
ekg = ekg.fillna(0).drop('ekg', axis=1)
ekg.head(2)

,id,ekg_date,ekg_sinus,ekg_ss,ekg_eos,ekg_qrs,ekg_qrst,ekg_pq
0,0,02.12.2016,1,120,1,0.07,0.26,0.12
1,1,13.01.2017,1,122,3,0.06,0.00,0.12


In [36]:
# qq = ekg[['ekg_date','id']].copy()
# qq['new_date'] = pd.to_datetime(ekg['ekg_date'])
# qq['op_d'] = pd.to_datetime(df['DateOfOper'])
# qq['new_date'] = qq['new_date'].fillna(qq['op_d'])
# qq['diff'] = (qq['op_d'] - qq['new_date']).dt.days
# qq['diff'].value_counts()

________________________________
## Opertatzii

In [37]:
opr = df[['id', 'first_op','target']].copy()
opr = opr.dropna()
opr['first_op'] = opr['first_op'].str.lower() \
                                 .str.replace('\.',' ', regex=True)\
                                 .str.replace('[^a-zа-я ]','',regex=True)

In [38]:
opr['first_op'] = opr['first_op'].apply(lambda x: ' '.join([w.lemma_ for w in nlp(x) if not w.is_stop]))

In [39]:
opr_t = opr[opr['target'] == 1]
opr_t = set(' '.join(opr_t['first_op'].values).split(' '))

In [40]:
opr_r = opr[opr['target'] != 1]
opr_r = set(' '.join(opr_r['first_op'].values).split(' '))

In [41]:
oper_set = opr_t & opr_r

In [42]:
# sum(opr_r['opertatzii'].str.contains('эмболиз'))
# opr_t['opertatzii'].str.contains('эмболиз') 
# opr_r & opr_t
# opr_t - opr_r
# opr_r - opr_t
# qq = opr['opertatzii'].str.replace(r'\(.*\)','', regex=True).str.lower().str.replace(r'[^а-я; ]','',regex=True) \
#.str.replace(' +',' ',regex=True).str.replace('; ',';', regex=True).str.strip().str.split(';')
# qq.explode().value_counts().head(30)

In [43]:
opr['first_op'] = opr['first_op'].map(lambda x: ' '.join(set(x.split(' ')) & oper_set))

In [44]:
opr = opr[['first_op', 'id']]
opr.columns = ['opr', 'id']

_____________________________________
## Merge

In [46]:
df = df.merge(medic_table, on='id', how='left')
df = df.merge(ekg, on='id', how='left')
df = df.merge(mkb, on='id', how='left')
df = df.merge(opr, on='id', how='left')
df = df.merge(eho, on='id', how='left')

In [47]:
#df['ekg_eos'] = df['ekg_eos'].fillna(' ')
#df['ekg_clr'] = df['ekg_clr'].fillna(' ').astype('str')
#df['first_op'] = df['first_op'].fillna(' ').astype('str')
df['opr'] = df['opr'].fillna(' ').astype('str')

In [48]:
df = df.fillna(0)

In [62]:
cols = ['drug_0', 'drug_1', 'drug_2', 'drug_3', 'drug_4', 'drug_5', 'drug_6', 'drug_7', 'drug_8', 'drug_9',
 'drug_10', 'drug_11', 'drug_12', 'drug_13', 'drug_14', 'drug_15', 'drug_16', 'drug_17', 'drug_18', 'drug_19']

text_features = ['opr']
cat_features = ['redo', 'sex', 'ekg_eos', 'ekg_sinus']

In [50]:
#df[cols] = df[cols].fillna(0).astype('int')
#df[cat_features] = df[cat_features].astype('str')

In [251]:
test_df = df[['sex','height','weight','BMI','BSA','age','target', 'redo', 'ekg_ss', 'ekg_sinus',
              'ekg_eos','ekg_qrs','ekg_qrst','ekg_pq', *mkb_list,  *cols]]

In [52]:
# train, test = train_test_split(test_df, test_size=0.4, random_state=2, stratify=test_df['target'])

# train_pool = Pool(train.drop('target', axis=1), 
#                  train['target'],
#                  cat_features=cat_features,
#                  text_features=text_features)

# valid_pool = Pool(test.drop('target', axis=1), 
#                  test['target'],
#                  cat_features=cat_features,
#                  text_features=text_features)
# model = CatBoostClassifier(random_seed=42, verbose=250, eval_metric='Recall', eta=0.5, iterations=500,
#                           auto_class_weights='Balanced'
#                           )
# model.fit(train_pool, eval_set=valid_pool, plot=True)

In [252]:
X = test_df.drop(['target'], axis=1)
y = test_df['target']

In [253]:
X.loc[X['redo'] == ' ', 'redo'] = 0

In [254]:
X['redo'] = X['redo'].astype('int')

In [277]:
X_train, X_ts, y_train, y_ts = train_test_split(X,y, test_size=0.3, random_state=1242, stratify=y)

In [270]:
ohe = make_column_transformer((OneHotEncoder(drop='first', handle_unknown='ignore'), cat_features),
                             # (TfidfVectorizer(), 'opr'),
                              remainder='passthrough')

ords = make_column_transformer((OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1),cat_features),
                              (TfidfVectorizer(), 'first_op'),
                              remainder='passthrough')

In [271]:
model = make_pipeline(ohe, 
                     # SMOTE(random_state=42), 
                      LogisticRegression(random_state=42, 
                                         penalty='l2',
                                         solver='liblinear', 
                                         max_iter=400,
                                         class_weight='balanced'
                                        ))

In [272]:
cross_val_score(model, X, y, cv=4, scoring='precision')

array([0.11111111, 0.33333333, 0.33333333, 0.28571429])

TEXT - array([0.14285714, 0.33333333, 0.5       , 0.2       ])  
NOTEXT - array([0.11111111, 0.33333333, 0.33333333, 0.28571429])

In [286]:
X_train, X_ts, y_train, y_ts = train_test_split(X,y, test_size=0.1, stratify=y)
model.fit(X_train, y_train)
m.confusion_matrix(y_ts, model.predict(X_ts))

array([[28,  1],
       [ 0,  1]])